In [1]:
!pip install requests playwright beautifulsoup4
!playwright install

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
123.2 Mb [                    ] 0% 0.0s123.2 Mb [                    ] 0% 136.1s123.2 Mb [                    ] 0% 286.6s123.2 Mb [                    ] 0% 266.0s123.2 Mb [                    ] 0% 353.5s123.2 Mb [                    ] 0% 272.4s123.2 Mb [                    ] 0% 432.8s123.2 Mb [                    ] 0% 435.6s123.2 Mb [                    ] 0% 415.6s123.2 Mb [                    ] 0% 427.8s123.2 Mb [                    ] 0% 413.5s123.2 Mb [                    ] 0% 460.4s123.2 Mb [                    ] 0% 425.1s123.2 Mb [                    ] 0% 411.9s123.2 Mb [                    ] 0% 431.5s123.2 Mb [                    ] 0% 474.5s123.2 Mb [                    ] 0% 405.6s123.2 Mb [                    ] 0% 396.9s123.2 Mb [                    ] 0% 406.0s123.2 Mb [                    ] 0% 428.3s123.2 Mb [

In [2]:
from playwright.sync_api import sync_playwright


def get_redbook_data(url):
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        page.goto(url)

        # 等待数据加载
        page.wait_for_selector('div.note-content')

        # 获取接口数据（需自行抓取实际接口）
        api_data = page.evaluate('''() => {
            return window.__initialData;
        }''')

        browser.close()
        return api_data

In [3]:
def parse_data(json_data):
    result = {
        "text": "",
        "images": []
    }

    # 提取正文
    if 'note' in json_data and 'content' in json_data['note']:
        result['text'] = json_data['note']['content']

    # 提取图片
    if 'image_list' in json_data.get('note', {}):
        for img in json_data['note']['image_list']:
            # 替换获取高清图（需根据实际数据结构调整）
            img_url = img['url'].replace('w/300/format', 'w/1080/format')
            result['images'].append(img_url)

    return result

In [4]:
url = "https://www.xiaohongshu.com/discovery/item/67b8a3aa000000000603b5b1?source=webshare&xhsshare=pc_web&xsec_token=ABl_iyad7-uPtWoRVhH2lcJ95dF61lI7QBOu1RNiziQI4=&xsec_source=pc_share"
raw_data = get_redbook_data(url)
parsed_data = parse_data(raw_data)

print("正文内容：", parsed_data['text'])
print("图片列表：", parsed_data['images'])

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.